In [307]:
import pandas as pd
import datetime as datetime

In [308]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}


In [309]:
def read_covid_time_series_data():
    return pd.read_csv('../../data/jhu/time_series_covid19_US.csv', parse_dates=['date'])

In [310]:
def create_year_month_column(df):
    df['yyyy-mm'] = df['date'].dt.strftime('%Y-%m')
    return df

In [311]:
def create_groupby_state_month(df):
    return df.groupby(['yyyy-mm', 'state']).last()['cases_conf_probable'].reset_index()

In [312]:
def remove_early_2020(df):
    return df.loc[~df['yyyy-mm'].isin(['2020-01', '2020-02']), :]

In [313]:
def clean_us_population(df):
    df.columns = ['state', 'population']
    df.iloc[:, 0] = df.iloc[:, 0].str[1:]
    df.iloc[:, 1] = pd.to_numeric(df.iloc[:, 1])
    return df

In [314]:
def read_us_population():
    df = pd.read_csv('../../data/us_population/nst-est2020.csv', header=None, thousands=',')
    df = clean_us_population(df)
    return df

In [315]:
def change_full_state_to_abbrev(df):
    for key, value in us_state_abbrev.items():
        df['state'].replace(key, value, inplace=True)
    return df

In [316]:
def normalize_state_by_population(pop_df, group_df):
    merged_df = group_df.merge(pop_df, on='state')
    merged_df['positive_cases_normalized'] = merged_df['cases_conf_probable'] / merged_df['population']
    return merged_df

In [317]:
data = read_covid_time_series_data()

In [318]:
data = create_year_month_column(data)

In [319]:
group_by_state_month = create_groupby_state_month(data)

### choose March 2020 - March 2021

In [320]:
group_by_state_month.groupby('yyyy-mm').sum()

,cases_conf_probable
yyyy-mm,
2020-01,2.0
2020-02,18.0
2020-03,196814.0
2020-04,1073152.0
2020-05,1791343.0
2020-06,2623024.0
2020-07,4523187.0
2020-08,5980439.0
2020-09,7173102.0


In [321]:
group_by_state_month = remove_early_2020(group_by_state_month)

### Make new cases column

In [322]:
def create_new_monthly_cases(df):
    df = df.sort_values(by=['state', 'yyyy-mm']).reset_index(drop=True)
    df['new_cases_monthly'] = df['cases_conf_probable'].diff()
    return df


In [323]:
group_by_state_month = create_new_monthly_cases(group_by_state_month)

In [324]:
group_by_state_month

,yyyy-mm,state,cases_conf_probable,new_cases_monthly
0,2020-03,AK,128.0,NaN
1,2020-04,AK,355.0,227.0
2,2020-05,AK,463.0,108.0
3,2020-06,AK,944.0,481.0
4,2020-07,AK,2993.0,2049.0
...,...,...,...,...
723,2020-11,WY,33305.0,20007.0
724,2020-12,WY,44409.0,11104.0
725,2021-01,WY,51912.0,7503.0
726,2021-02,WY,54394.0,2482.0


In [326]:
def get_row_of_negative(df):
    '''checking which rows in the groupings are negative'''
    month_where_negative = list()
    for state in df.state.unique():
        month_where_negative.append(df.loc[(df['state'] == state) \
                                   & (df['new_cases_monthly'] < 0), 'yyyy-mm'].values)
    return month_where_negative

#get_row_of_negative(group_by_state_month)

# groupings seem to be the first row in year and monthly data

In [332]:
def make_first_row_cumulative_case(df):
    for state in df.state.unique():
        value_to_change = df.loc[df['state'] == state, 'cases_conf_probable'].head(1)
        df.loc[df['state'] == state, 'new_cases_monthly'].iloc[0,] = value_to_change
    print(type(value_to_change))
    return df

In [333]:
group_by_state_month = make_first_row_cumulative_case(group_by_state_month)

<class 'pandas.core.series.Series'>


In [335]:
group_by_state_month[group_by_state_month['state'] == 'VA']

,yyyy-mm,state,cases_conf_probable,new_cases_monthly
637,2020-03,VA,1250.0,-377831.0
638,2020-04,VA,15846.0,14596.0
639,2020-05,VA,44607.0,28761.0
640,2020-06,VA,62787.0,18180.0
641,2020-07,VA,89888.0,27101.0
642,2020-08,VA,120594.0,30706.0
643,2020-09,VA,148271.0,27677.0
644,2020-10,VA,181190.0,32919.0
645,2020-11,VA,237835.0,56645.0
646,2020-12,VA,349584.0,111749.0


### Normalize by the population of each state

In [118]:
population = read_us_population()

In [125]:
abbrev_population = change_full_state_to_abbrev(population)

In [126]:
abbrev_population

,state,population
0,AL,4921532
1,AK,731158
2,AZ,7421401
3,AR,3030522
4,CA,39368078
5,CO,5807719
6,CT,3557006
7,DE,986809
8,DC,712816
9,FL,21733312


In [131]:
normalized_df = normalize_state_by_population(abbrev_population, group_by_state_month)

In [135]:
normalized_df.sort_values(by='positive_cases_normalized', ascending=False)

,yyyy-mm,state,cases_conf_probable,population,positive_cases_normalized
376,2021-03,ND,101150.0,765309,0.132169
375,2021-02,ND,99809.0,765309,0.130417
545,2021-03,SD,114791.0,892717,0.128586
374,2021-01,ND,97630.0,765309,0.127569
544,2021-02,SD,112427.0,892717,0.125938
...,...,...,...,...,...
533,2020-03,SD,108.0,892717,0.000121
559,2020-03,TX,3266.0,29360759,0.000111
221,2020-03,KY,480.0,4477251,0.000107
637,2020-03,WV,162.0,1784787,0.000091
